# OSU Energy Dataset Starter Notebook (Spark SQL)

### **Data Use Notice**
This dataset is sourced from Ohio State’s public dashboard and is being provided for use **only within the Data I/O 2026 challenge.** 

By participating, you agree to follow [Ohio State’s IDP policy](https://it.osu.edu/data/institutional-data-policy) and understand that this data should **not be used or shared outside of this competition.**

**Instructions:**
1. Run the **Install & Download Data** cell first.
2. Then run the rest of the code to load CSVs into Spark SQL temporary views.
3. Table names are based on CSV filenames, with hyphens replaced by underscores.
4. Query a table with `spark.sql("SELECT * FROM <table_name>")`.


In [0]:
# ---------------------------
# Install gdown
# ---------------------------
%pip install gdown --quiet

import gdown
import zipfile
import pandas as pd
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder.getOrCreate()

WORKSPACE_USER = "anshumanr10@gmail.com"
if os.environ.get("DATABRICKS_RUNTIME_VERSION"):
    workspace_folder = "/tmp/energy_dataset"
else:
    workspace_folder = os.path.join(os.getcwd(), "data")
os.makedirs(workspace_folder, exist_ok=True)

# ---------------------------
# Step 1: Download Core + Bonus ZIPs
# ---------------------------
zip_files = {
    "core": "https://drive.google.com/uc?id=13o_2ojFRCCqwmYMN3w3qu5fQxieXATTd",
    "bonus": "https://drive.google.com/uc?id=1Hvqi5nv66m3b1aEN23NnUOBkVKQrfP5z"
}

extracted_csv_paths = []

for name, url in zip_files.items():
    zip_path = os.path.join(tmp_folder, f"{name}_dataset.zip")
    print(f"\nDownloading {name} ZIP...")
    gdown.download(url, zip_path, quiet=False)
    
    print(f"Extracting CSVs from {name} ZIP...")
    with zipfile.ZipFile(zip_path, "r") as z:
        for member in z.namelist():
            if member.endswith(".csv") and "__MACOSX" not in member:
                print(f"  Extracting {member}")
                z.extract(member, tmp_folder)
                extracted_csv_paths.append(os.path.join(tmp_folder, member))

# ---------------------------
# Step 2: Print list of CSV files
# ---------------------------
print("\nAll extracted CSV files:")
for csv_path in extracted_csv_paths:
    print(f" - {os.path.basename(csv_path)}")

# ---------------------------
# Step 3: Load CSVs into Spark SQL
# ---------------------------
for csv_path in extracted_csv_paths:
    csv_name = os.path.basename(csv_path)
    print(f"\nLoading {csv_name} into Spark SQL view...")
    
    pdf = pd.read_csv(csv_path, encoding="latin1")
    df = spark.createDataFrame(pdf)
    
    view_name = os.path.splitext(csv_name)[0].replace("-", "_")
    df.createOrReplaceTempView(view_name)
    
    print(f"Temp view created: {view_name}, rows: {df.count()}")

# Example usage
first_view = os.path.splitext(os.path.basename(extracted_csv_paths[0]))[0].replace("-", "_")
spark.sql(f"SELECT * FROM {first_view} LIMIT 5").show()


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.



Downloading...
From (original): https://drive.google.com/uc?id=13o_2ojFRCCqwmYMN3w3qu5fQxieXATTd
From (redirected): https://drive.google.com/uc?id=13o_2ojFRCCqwmYMN3w3qu5fQxieXATTd&confirm=t&uuid=a2a2137c-1dd7-4de6-9d38-fe8420fb6244
To: /tmp/energy_dataset/core_dataset.zip
100%|██████████| 201M/201M [00:01<00:00, 104MB/s]


Extracting CSVs from core ZIP...
  Extracting advanced_core/meter-readings-march-2025.csv
  Extracting advanced_core/meter-readings-april-2025.csv
  Extracting advanced_core/weather_data_hourly_2025.csv
  Extracting advanced_core/meter-readings-jan-2025.csv
  Extracting advanced_core/building_metadata.csv
  Extracting advanced_core/meter-readings-feb-2025.csv



Downloading...
From (original): https://drive.google.com/uc?id=1Hvqi5nv66m3b1aEN23NnUOBkVKQrfP5z
From (redirected): https://drive.google.com/uc?id=1Hvqi5nv66m3b1aEN23NnUOBkVKQrfP5z&confirm=t&uuid=34c010be-e83c-4a9a-a551-0ec9ff6ebfb0
To: /tmp/energy_dataset/bonus_dataset.zip
100%|██████████| 416M/416M [00:03<00:00, 127MB/s]


Extracting CSVs from bonus ZIP...
  Extracting advanced_bonus/meter-readings-may-2025.csv
  Extracting advanced_bonus/meter-readings-sept-2025.csv
  Extracting advanced_bonus/meter-readings-nov-2025.csv
  Extracting advanced_bonus/meter-readings-dec-2025.csv
  Extracting advanced_bonus/meter-readings-aug-2025.csv
  Extracting advanced_bonus/meter-readings-oct-2025.csv
  Extracting advanced_bonus/meter-readings-june-2025.csv
  Extracting advanced_bonus/meter-readings-july-2025.csv

All extracted CSV files:
 - meter-readings-march-2025.csv
 - meter-readings-april-2025.csv
 - weather_data_hourly_2025.csv
 - meter-readings-jan-2025.csv
 - building_metadata.csv
 - meter-readings-feb-2025.csv
 - meter-readings-may-2025.csv
 - meter-readings-sept-2025.csv
 - meter-readings-nov-2025.csv
 - meter-readings-dec-2025.csv
 - meter-readings-aug-2025.csv
 - meter-readings-oct-2025.csv
 - meter-readings-june-2025.csv
 - meter-readings-july-2025.csv

Loading meter-readings-march-2025.csv into Spark SQL